# Purpose of This Project
This project will contain python scripts for the purposes of simulating and analyzing GURPS 4e combat mechanics. I plan to take a functional approach to this calculator to practice my functional programming in python. I may or may not break down and cry and write for loops instead. I expect to embarrassingly mess up the whole adhering to the functional paradigm entirely and have to revisit a lot of my functions. Either way, I'll write it all down in a Jupyter notebook to document the journey.

First, I'll start by finding a function that can roll dice. GURPS 4e combat is highly reliant on dice rolls, so—while we can analyze the rolls mathematically—the ability to run simulations should be a lot more practical in situations where we are learning the mathematics or want to see how it might feel to a player in our game.

We'll use the random module

In [1]:
import random as rd

print(rd.randint(3,18))

6


GURPS 4e heavily utilizes the roll of three six sided dice (3d6), abbreviated in the context of the system simply to 3d. Our 3d rolls will have a minimum of 3 and a maximum of 18, as shown above, but the distribution won't be completely linear. We might repackage the `rd.randint()` function into another function. On that note, regarding the functional approach I'd like to take, I am not going to remake `rd.randint()` from scratch to make it more "functional". This approach practice will only apply to user-defined functions for the time being.

First, we should aim to make a couple of functions: one to roll any number of dice and output a number, particularly rolls of 3d, and one that calculates the probability of getting any given number from a roll.

We are going to assume that every single dice we work with will be one that starts at 1, and counts up one whole number until there is a unique number on every face of an n-sided die from 1 to n.

# Dice Roll Functions

In [2]:
def roll_dx(x):
    return rd.randint(1,x)

def roll_xd6(x):
    return x*roll_dx(6)

print(f'roll 3d6 5 times: {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}')
print(f'roll 5d6 5 times: {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}')

roll 3d6 5 times: 6, 12, 9, 9, 12
roll 5d6 5 times: 10, 15, 15, 20, 30


The way that our `roll_dx()` function is written, it will give an error for any input besides whole numbers greater than or equal to `1`.

# Dice Probability Functions

Our first dice probability function will be a lot like our function `roll_dx` above. The input for the function will be a whole number greater than or equal to `1` that says how many sides we have on our dice. The output will be a dictionary of all the possible outcomes as keys and their probabilities as values.

## Coming up with a solution
the first approach that comes to mind is to take our aforementioned input as a local variable `x`, then create an empty dictionary to add entries to. In the case of an input that is `6`, we would want our function to add numbers from 1 to 6 to our dictionary, and make the value a probability in the form of a calculation: 1/6 for each one.

I'm doing some lame empty dict technique for this, so it may seem like I have given up on making this hardcore functional and you are correct. The reason is that I have a baby to take care of and she doesn't have the patience for dad to learn how to do this. Refactoring our functions will come at a later date.

In [3]:
def prob_dx(x):
    d = {}
    for i in range(x):
        d[i+1] = 1/x
    return d

print(prob_dx(6))

{1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666}


Supposedly it is ok to use a dictionary and return one, as long as we don't store it anywhere, creating some mutable state. We could force the output of the function to be an iterator, but I think we are adults and can decide whether or not we want to use our functions as functionally as possible.

# WHAT IS FUNCTIONAL PROGRAMMING

For the amount I've been mentioning functional programming, I sure haven't defined what it is.

For the purposes of this project, we will be using [this](https://docs.python.org/3/howto/functional.html) page as a standard for what functional programming in python is. This howto emphasizes modularity, composability, and ease of debugging and testing.

# Dice Probability Functions cont.

Let's try to refactor our function from above with a dictionary comprehension instead of all the lines we used

In [4]:
def prob_dx(x):
    return {n+1:1/x for n in range(x)}

print(prob_dx(6))

{1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666}


nice and sleek

Next, we'll want something that can give us the probability distribution of multiple dice added together in the same roll, such as 3d6. The resulting dictionary in the case of 3d6 should have values 3 through 18 and their probabilities as a number from 0 to 1. If we wanted to calculate this, we might list the number of permutations possible with 3d6 and figure out how many combinations out of 216 give a certain value.

I have a feeling that, instead of doing that, we can write a function that does this on the fly while doing math with dictionaries, so let's try.

## Finding a solution for 3d6 probability calculator

We would want to add up our keys to get new keys: 1+1+1=3, 1+1+2=4, etc. sounds like for loop hell that I may or may not refactor into a comprehension.

How do the values factor in? well, in the case of 1+1+1=3, we want to end up doing some operation with 1/6, 1/6, and 1/6 that comes out to 1/216 or 1/(6^3). It seems obvious that we will want to multiply our probabilities together. Let's check 1+1+2: that gives the same exact result, but our for loop hell will run into 4 examples of this. How do we add the probabilities together when we calculate a key that is already in use?

We can reference the previous value for the key and add our result to it to set the value of the key. This is poopy mutability, but I'll survive for now. I just have to figure out how to make this work for setting the key in the first place.

`dict[1+1+1] += 1/6 * 1/6 * 1/6` works if `dict[1+1+1]` already equals 0. But IDK if that works of if I have to set the key value to 0 first.

...

I just checked and yeah you can't set a dictionary entry with `+=`

so one solution is to do an if statement to check if the key exists or not, and the other is to do a for loop hell before this one to set up the keys for the dictionary. Both of these suck.

After much searching I found the .get() method for dictionaries. It is identical to using a key as an index except you can't assign anything to it, and, importantly, you can set a default value for keys that don't exist yet. So that gives me a great way to do something like += except being able to start off without a key via default=0.

In [5]:
def prob_d6():
    return prob_dx(6)

def prob_3d6():
    d = {}
    for n, m in prob_d6().items():
        for j, k in prob_d6().items():
            for i, o in prob_d6().items():
                d[n+j+i] = d.get(n+j+i, 0) + (m*k*o)
    return d
                
print(prob_3d6())

{3: 0.004629629629629629, 4: 0.013888888888888888, 5: 0.027777777777777776, 6: 0.046296296296296294, 7: 0.06944444444444445, 8: 0.09722222222222218, 9: 0.11574074074074067, 10: 0.12499999999999992, 11: 0.12499999999999992, 12: 0.11574074074074067, 13: 0.09722222222222218, 14: 0.06944444444444445, 15: 0.046296296296296294, 16: 0.027777777777777776, 17: 0.013888888888888888, 18: 0.004629629629629629}


Two things:
1. Holy floats batman, I'm sure there is some kind of fractions library I can use to make this a lot cleaner
2. for loop hell is looking really shabby too, I'll have to look into those fancy functional functions like filter. It's worth noting that the reason for the for loops is for finding permutations and calculating probabilities together all in one step. This could have been a lot more annoying; I hate looking for the permutations of big dice rolls that are totaled together.
Bonus thing 3. This function could be generalized. I don't know how, maybe one of those magical functional functions, but I'm sure it can be. Maybe recursion

Anyway, let's solve problem number 1

## Representing probabilities as fractions

the original fractions come from our `prob_dx` function from earlier, so we'll focus on that.

In [6]:
from fractions import Fraction

def prob_dx(x):
    return {n+1:Fraction(1, x) for n in range(x)}

print(prob_dx(6))

{1: Fraction(1, 6), 2: Fraction(1, 6), 3: Fraction(1, 6), 4: Fraction(1, 6), 5: Fraction(1, 6), 6: Fraction(1, 6)}


Cool, let's see if it works with no changes for `prob_3d6`

In [7]:
print(prob_3d6())

{3: Fraction(1, 216), 4: Fraction(1, 72), 5: Fraction(1, 36), 6: Fraction(5, 108), 7: Fraction(5, 72), 8: Fraction(7, 72), 9: Fraction(25, 216), 10: Fraction(1, 8), 11: Fraction(1, 8), 12: Fraction(25, 216), 13: Fraction(7, 72), 14: Fraction(5, 72), 15: Fraction(5, 108), 16: Fraction(1, 36), 17: Fraction(1, 72), 18: Fraction(1, 216)}


That is fantastic. If we want, we can display our values as more accurate decimal representations, and we'll want to in order to analyze probabilities, but I think better in terms of fractions for sure, so we'll leave it for now.

## Generalize `prob_3d6` into `prob_xd6`

The idea right now is to use recursion to generalize our nested for loops in order to nest an arbitrary number of times.

Let's start backwards: the final line—`d[n+j+i] = d.get(n+j+i, 0) + (m*k*o)`—can be generalized. Let's try using the `sum()` function and the `numpy.prod()` function.

`d[sum(list_of_x_keys)] = d.get(sum(list_of_x_keys), 0) + np.prod(list_of_x_values)`

should work!

This means that we need to define those lists at some point. We are going to nest x for loops together—actually maybe something in itertools should do this? I realized I was about to use the word iterate, and I was gazing at the `itertools.Product()` function wondering what it was for while I was looking for `numpy.prod()`, and it looks like it might be a fit for this? if not there are many other options, so I will explore [that](https://docs.python.org/3/library/itertools.html) later.

It looks like, researching itertools, I can use a combination of my `prod_d6()` function—or some slight modification of it—with the `itertools.accumulate()` function. I'd use the default summing for the dictionary keys and something like `numpyp.prod()` within `itertools.accumulate()` for the dictionary values

It's a bit to wrap my head around though.

The way I did it earlier does the cartesian product that indicates all possible rolls, the totaling of the rolls as one would do in a game setting, and the probability calculation all in one step. Every time I try to write code using itertools functions to do what I did earlier for `prob_3d6`, I get close, but I don't see how to combine everything to get the result I want.

### Brainstorm approaches

Approach 1: nested for loops set up three dictionaries and math is done to calculate roll totals and probabilities all at once.

Approach 2: calculate cartesian product, sum all combinations listed by the cartesian product and end up with an iterable that looks like `[3, 4, 4, 4, etc.]`, calculate the probability of a unique dice roll (ex. 1/216) either by taking one divided by the length of the iterable or by doing one divided by the product of the number of sides of each dice, and then finally get the probability of any unique roll total by taking how many times the roll total appears in the list and multiplying it by my probability

Approach 1.5: generalize nested for loops and use the approach I already have worked out? It really seems to be the more beautiful solution.

Approach 1.1: write a list comprehension version of what I already have?

### Brainstorm 2

* number of dice being totaled together, x
* number of faces on dice, y (6 by default)
* probability of a total occurring, p = y**-x
* `itertools.product(range(1, y+1), repeat=x)`
* sum all 216 items to find totals
* count all 216 items and return how many of each total there is, like `Series.value_counts()` from Pandas
* take this list of numbers and how often they appear and multiply the frequency by p
* the list is now probabilities for any given total

In [8]:
import itertools as it
import pandas as pd

def prob_nds(num_of_dice=3, sides=6):
    all_possible_totals = pd.Series(sum(element) for element in it.product(range(1, sides+1), repeat=num_of_dice))
    return all_possible_totals.value_counts(sort=False) * sides**-num_of_dice

prob_nds()

3     0.004630
4     0.013889
5     0.027778
6     0.046296
7     0.069444
8     0.097222
9     0.115741
10    0.125000
11    0.125000
12    0.115741
13    0.097222
14    0.069444
15    0.046296
16    0.027778
17    0.013889
18    0.004630
dtype: float64

## OOF, Finally
That took too long! Functional paradigms feel like they are gone out the window, assuming we even knew what they were to begin with. There may be a way to get rid of that for loop that is used for the generator expression, we probably don't need this to be a pandas Series in the slightest but I don't know an easy way to do it otherwise, and who knows how it compares to our previous function performance-wise, but I'm mostly just glad that it works.

### Further usefulness (probably for a much later date):
* Be able to calculate and display probabilities as the probability of getting a certain result or lower/or higher
* Calculate probability for totals of mixed dice types, i.e. d10 + 2d8 (not that this is used in any game system, but it might be cool if it were used in some system, may be a cool experimental game design tool!)
* Account for non-standard-faced dice like [Grime dice](https://mathsgear.co.uk/products/non-transitive-grime-dice) or dice that start at 0 or 2 or whatever
* Account for dice where low results like 1s are rerolled, but this is covered in the last point since a d6 roll that rerolls on a 1 is a d5 that starts at 2.
* Account for rolls like 4d6 drop lowest.

## Moving forward
We'll start working towards calculating DPS—damage per second—in GURPS combat.

Doing so can be a bit complicated, because there is a hit chance which determines if you are allowed to do damage, a defense chance which determines again whether or not you are allowed to do damage, and then chance involved in determining the actual damage value. Furthermore there are combat choices that can trade off hit chance for defense chance, hit chance for damage amount, hit chance for more attack attempts, and critical hit bonuses. On top of that, there are strategies like striking the head that—depending on the dice—could result in incapacitation and thus very high DPS due to low defense chance, attacking with multiple people per round on one target which can affect defense chance, and even non-direct damage strategies like trying to push a target off of a cliff. Furthermore, a lot of the attributes invovled in these calculations are often weighted by a point-buy system. Again, calculating DPS can be a bit complicated.

DPS is often more useful in long fights, but GURPS combat tends to be volatile and over quickly. The value of a DPS number in this case is as an indicator of what is the safest method to deal damage—the optimal gambling strategies.

# DPS calculations, preliminary work

Now that we have something that can roll dice and tell us the probability of getting a given total—a set of faces added together—we can hopefully make good use of it by giving context to the damage rolls in GURPS, and use it to measure the trade-offs between various combat options in the system.

We'll start by seeing if we can code a function that tells us DPS in a very contrived situation: There is an attack roll with an effective skill at 14, a defense roll with an effective skill at 9, and a damage roll of 1d of crushing damage. What is the DPS?

Before the coding though, let's see if we can intuit the answer. Assuming just the damage roll, the DPS is 3.5 DPS (one attack roll per round, and one round takes one second in GURPS 4e. If we add in a defense roll, we realize that we have the probability for any given total, but what we actually want is the probability of getting 9 or lower. Let's code that really quick.

In [9]:
def prob_success_3d6():
    return prob_nds(num_of_dice=3, sides=6).cumsum()
    
prob_success_3d6()

3     0.004630
4     0.018519
5     0.046296
6     0.092593
7     0.162037
8     0.259259
9     0.375000
10    0.500000
11    0.625000
12    0.740741
13    0.837963
14    0.907407
15    0.953704
16    0.981481
17    0.995370
18    1.000000
dtype: float64

Another issue to worry about in the future has to do with index 18. It is a given that aiming for a total of 18 or lower (meaning a roll for skill at 18) has a 100% success rate, but this is notable for a special reason, because an 18 is a "critical fail" in GURPS regardless of almost any other factors; combined with a critical fail table, this makes a roll of 18 a can of worms for calculating DPS. There are similar issues like 3 and 4 being a "critical success", accompanied by at least two tables; additionally, 17 is an automatic fail, and there are conditions that widen the critical success range. We'll cross that bridge when we get there.

Now, back to our contrived situation: the first roll is an attack roll at skill 14, so there is a 91% chance of that succeeding. Then there is a defense roll; we actually want to know the chance of this failing, so we'll make another function that does a reverse cumulative sum.

In [10]:
def prob_failure_3d6():
    return prob_nds(num_of_dice=3, sides=6)[::-1].cumsum()

prob_failure_3d6()

18    0.004630
17    0.018519
16    0.046296
15    0.092593
14    0.162037
13    0.259259
12    0.375000
11    0.500000
10    0.625000
9     0.740741
8     0.837963
7     0.907407
6     0.953704
5     0.981481
4     0.995370
3     1.000000
dtype: float64

The way that we want to read this is that a dodge roll at skill 3 has a 100% chance of failure. This is not true.

Our previous function gives us the probability of getting a certain total or lower. This function gives us the probability of getting a certain total or higher, but we don't want that because a getting a certain total is a success. We want the probability of getting lower than a certain total not including said certain total. We don't have any boolean statements going on so we can't fix our inequality operator. A way might be to subtract 1 from all of our indexes; this would include the row [2, 100% chance of failure], which is true in GURPS, and would include [18, 0% chance of failure], which is not true because of our special rules regarding 17 and 18. It will work well enough for now once we code it.

In [11]:
def prob_failure_3d6():
    subtract_one_from_index = pd.Series(data=prob_nds(num_of_dice=3, sides=6), index=prob_nds(num_of_dice=3, sides=6).index - 1)
    return subtract_one_from_index[::-1].cumsum()

prob_failure_3d6()

17    0.013889
16    0.041667
15    0.087963
14    0.157407
13    0.254630
12    0.370370
11    0.495370
10    0.620370
9     0.736111
8     0.833333
7     0.902778
6     0.949074
5     0.976852
4     0.990741
3     0.995370
2          NaN
dtype: float64

Something strange happened! Unfortunately, when we manipulated our indexes, our values changed to match them; the value for 18 is gone and the value for 2 doesn't exist. Maybe we can try to reset the indexes with a list comprehension instead.

In [12]:
def prob_failure_3d6():
    subtract_one_from_index = prob_nds(num_of_dice=3, sides=6)
    subtract_one_from_index.index = (index - 1 for index in subtract_one_from_index.index)
    return subtract_one_from_index[::-1].cumsum()

prob_failure_3d6()

17    0.004630
16    0.018519
15    0.046296
14    0.092593
13    0.162037
12    0.259259
11    0.375000
10    0.500000
9     0.625000
8     0.740741
7     0.837963
6     0.907407
5     0.953704
4     0.981481
3     0.995370
2     1.000000
dtype: float64

That is correct! woo

I used a generator expression instead of a list comprehension though; I had forgotten they existed temporarily. I guess you could say I was lazy or that it simply didn't take much space in my memory. We could have just taken the chance of success and subtracted it from 100%, but somehow I didn't think of that. At least we learned something about pandas indexes.

back to our combat situation: the first roll is an attack roll at skill 14 with a 90.7% chance of success. Then there is a defense roll at skill 9 with a 62.5% chance of failure, which in this case a failure is a "success" from the attacker's perspective, bringing the total probability of landing a hit to `prob_success_3d6().loc[14] * prob_failure_3d6().loc[9]`

In [13]:
prob_success_3d6().loc[14] * prob_failure_3d6().loc[9]

0.5671296296296295

which equals 56.7% chance to land a hit. The system seems to assume that PCs will be using skills at levels such as this, and people wonder why their GURPS combat seems sluggish. To make matters worse, effective defense skills tend to be even higher and damage often ends with damage reduction from armor. This might be something we can balance out if we analyze it with math!

Next, we need to see how a 56.7% chance to hit an average of 3.5 damage translates to DPS, then we can add other factors from the system that will complicate matters.

what does the math for this look like? Well, if we have a 0% chance to hit (ignoring critical success), we have 0 DPS, and if we have a 100% chance to hit (again ignoring critical success), then we have 3.5 DPS. It would seem that if we have a 50% chance, then the expectation is that our DPS is 3.5 one second, and 0 the next; there isn't anything weird going on here, to calculate DPS, we multiply the damage by the chance to hit.

When we make our function, we'll have to figure out what the inputs should be, but for now we'll just try and iterate as we go.

In [18]:
# A simple function that assumes a regular n sided dice with the faces labeled 1 through n, and returns the average roll.
# We'll use math instead of actually adding up the sides and taking an average
def average_damage(sides_on_dice=6):
    return sides_on_dice / 2 + 0.5

def dps(attack_skill, defense_skill, sides_on_dice=6):
    return prob_success_3d6().loc[attack_skill] * prob_failure_3d6().loc[defense_skill] * average_damage(sides_on_dice)

print(prob_success_3d6().loc[14] * prob_failure_3d6().loc[9])
print(dps(14, 9))

0.5671296296296295
1.9849537037037033


With an attack skill of 14, a defense skill of 9, and a damage dice of 1d crushing damage, our damage per second is expected to be a little less than 2 damage per second. With no outstanding factors and no critical failures or critical successes, we can expect combat against a 10 HP target to take about 5 rounds, of course an outstanding factor that naturally arises here is that when a target is at low HP, they can become worse at various defenses such as dodging. Again, right now the goal with our `dps()` function is to provide a heuristic for what action is the most likely to progress combat on a turn to turn basis, and how the game will feel to a player. We learned that about half of the turns taken will result in nothing happening, and that it will take an average of 5 rounds, though in the future we might want to run simulations to see if the average is a useful value or if a large percentage of simulations take 4 or 6 rounds or some result that splits the average even more egregiously; in other words "how swingy is our combat scenario".

Another common trait to look forward to is damage reduction: we'll have to answer the question of whether damage reduction is commutative—whether we can just add it into our multiplication in our `dps()` function—or whether we'll need to factor it in in a different way.